In [1]:
!pip install linearmodels

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
pip install scipy==1.10.1 statsmodels==0.13.5


Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following yanked versions: 1.11.0, 1.14.0rc1
ERROR: Ignored the following versions that require a different python version: 1.10.0 Requires-Python <3.12,>=3.8; 1.10.0rc1 Requires-Python <3.12,>=3.8; 1.10.0rc2 Requires-Python <3.12,>=3.8; 1.10.1 Requires-Python <3.12,>=3.8; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11; 1.9.0 Requires-Python >=3.8,<3.12; 1.9.0rc1 Requires-Python >=3.8,<3.12; 1.9.0rc2 Requires-Python >=3.8,<3.12; 1.9.0rc3 Requires-Python >=3.8,<3.12; 1.9.1 Requires-Python >=3.8,<3.12
ERROR: Could not find a version that satisfies the requirement scipy==1.10.1 (from versions:

In [3]:
import pandas as pd
df = pd.read_csv("../data/processed/merged_panel_clean_data.csv")
# import pandas as pd

# # Load cleaned panel
# df = pd.read_csv("merged_panel_clean_data.csv")

# Create a combined time identifier
df["zeit"] = df["jahr"].astype(str) + "Q" + df["quartal"].astype(str)
df["zeit"] = df["zeit"].astype("category")

# Optional: define post-treatment indicator (cumulative)
df["post"] = df.groupby("kasse_clean")["treatment_flag"].transform(lambda x: x.cumsum() > 0)

# print(df.columns.tolist())
# import statsmodels.formula.api as smf

df["treatment_flag"] = df["treatment_flag"].astype(int)
df["post"] = df["post"].astype(int)
df["churn_rate"] = pd.to_numeric(df["churn_rate"], errors="coerce")
df["morbidity_index"] = pd.to_numeric(df["morbidity_index"], errors="coerce")
df["zusatzbeitrag_lag"] = pd.to_numeric(df["zusatzbeitrag_lag"], errors="coerce")

df_model = df.dropna(subset=["churn_rate", "treatment_flag", "post", "morbidity_index", "zusatzbeitrag_lag"])

# # Fit OLS model with interaction
# did_model = smf.ols(
#     formula="churn_rate ~ treatment_flag * post + morbidity_index + zusatzbeitrag_lag",
#     data=df
# ).fit()

# # Show results
# print(did_model.summary())

import statsmodels.formula.api as smf

did_model = smf.ols(
    formula="churn_rate ~ treatment_flag * post + morbidity_index + zusatzbeitrag_lag",
    data=df_model
).fit()

print(did_model.summary())


C:\Users\Vanshika\AppData\Local\Temp\ipykernel_11524\58881669.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/merged_panel_clean_data.csv")


                            OLS Regression Results                            
Dep. Variable:             churn_rate   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     244.1
Date:                Tue, 01 Jul 2025   Prob (F-statistic):          1.04e-154
Time:                        18:11:35   Log-Likelihood:                 53594.
No. Observations:               15202   AIC:                        -1.072e+05
Df Residuals:                   15198   BIC:                        -1.072e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.0035    

In [5]:
# Install required packages if you haven't yet
# !pip install linearmodels statsmodels

import pandas as pd
from linearmodels.panel import PanelOLS

# Load cleaned panel data
df = pd.read_csv("../data/processed/merged_panel_clean_data.csv")

# Create combined time identifier (e.g., "2023Q1")
df["zeit"] = df["jahr"].astype(str) + "Q" + df["quartal"].astype(str)

# Convert "zeit" to a pandas Period and then to Timestamp for proper panel indexing
df["zeit_dt"] = pd.PeriodIndex(df["zeit"].astype(str), freq="Q").to_timestamp()

# Convert fund ID to string (entity id)
df["kasse_clean"] = df["kasse_clean"].astype(str)

# Define post-treatment indicator (cumulative sum > 0 per fund)
df["post"] = df.groupby("kasse_clean")["treatment_flag"].transform(lambda x: x.cumsum() > 0).astype(int)

# Convert treatment_flag to int if not already
df["treatment_flag"] = df["treatment_flag"].astype(int)

# Ensure numeric columns are properly typed (coerce errors to NaN)
df["churn_rate"] = pd.to_numeric(df["churn_rate"], errors="coerce")
df["morbidity_index"] = pd.to_numeric(df["morbidity_index"], errors="coerce")
df["zusatzbeitrag_lag"] = pd.to_numeric(df["zusatzbeitrag_lag"], errors="coerce")

# Drop rows with missing values in relevant columns
df_model = df.dropna(subset=["churn_rate", "treatment_flag", "post", "morbidity_index", "zusatzbeitrag_lag"])

# Set multi-index for panel data (entity: kasse_clean, time: zeit_dt)
df_model = df_model.set_index(["kasse_clean", "zeit_dt"])

# Specify the model formula:
# We model churn_rate as a function of treatment_flag, morbidity_index, zusatzbeitrag_lag
# + fixed effects for entity and time
model = PanelOLS.from_formula(
    formula="churn_rate ~ treatment_flag + morbidity_index + zusatzbeitrag_lag + EntityEffects + TimeEffects",
    data=df_model
)

# Fit the model
fe_results = model.fit()

# Print the summary of fixed effects DiD model
print(fe_results.summary)

C:\Users\Vanshika\AppData\Local\Temp\ipykernel_11524\2762360817.py:8: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/merged_panel_clean_data.csv")


                          PanelOLS Estimation Summary                           
Dep. Variable:             churn_rate   R-squared:                        0.0126
Estimator:                   PanelOLS   R-squared (Between):             -2.4659
No. Observations:               15202   R-squared (Within):               0.0180
Date:                Tue, Jul 01 2025   R-squared (Overall):             -1.0699
Time:                        18:12:01   Log-likelihood                 5.462e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      64.009
Entities:                          93   P-value                           0.0000
Avg Obs:                       163.46   Distribution:                 F(3,15070)
Min Obs:                       4.0000                                           
Max Obs:                       561.00   F-statistic (robust):             64.009
                            